In [ ]:
# 🧠 Paket kurulumu
!pip install ultralytics geopandas shapely onnxruntime rasterio scikit-learn requests -q

# 🧩 stream_downloader.py modülünü oluştur
with open("stream_downloader.py", "w") as f:
    f.write("""
import requests

def download_streaming(url: str, output_path: str, chunk_size: int = 8192, verbose: bool = True) -> bool:
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(output_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:
                    file.write(chunk)
        if verbose:
            print(f"✅ Dosya başarıyla indirildi: {output_path}")
        return True
    except Exception as e:
        if verbose:
            print(f"⛔ İndirme hatası: {e}")
        return False
""")

# 🔗 Modülü içeri aktar
from stream_downloader import download_streaming

# 📁 Modüller
import os, shutil, requests, tarfile
import rasterio
from rasterio.plot import reshape_as_image
from PIL import Image
import numpy as np
import geopandas as gpd
from sklearn.model_selection import train_test_split
import onnxruntime
from ultralytics import YOLO

# 📦 Yol tanımları
zip_url = "https://spacenet-dataset.s3.amazonaws.com/spacenet/SN2_buildings/tarballs/SN2_buildings_train_AOI_2_Vegas.tar.gz"
local_path = "vegas_buildings.tar.gz"
extract_dir = "./spacenet"

# 🗑️ Ön temizlik
for path in [local_path, extract_dir]:
    if os.path.exists(path):
        if os.path.isfile(path):
            os.remove(path)
        else:
            shutil.rmtree(path)

if os.path.exists("runs/detect"):
    for item in os.listdir("runs/detect"):
        if item.startswith("train"):
            shutil.rmtree(os.path.join("runs/detect", item))

# 📥 Dosyayı indir
print("📦 Dosya parça parça indiriliyor...")
success = download_streaming(zip_url, local_path)

# 🗜️ Arşiv çıkarma
if success and os.path.exists(local_path):
    os.makedirs(extract_dir, exist_ok=True)
    with tarfile.open(local_path, "r:gz") as tar:
        tar.extractall(path=extract_dir)
    print("🗜️ Arşiv başarıyla çıkarıldı.")
else:
    print("❌ Dosya indirilemedi veya bulunamadı.")

# 🔍 Klasör doğrulama
for root, _, files in os.walk(extract_dir):
    print(root, len(files), "dosya")

# 🎨 GeoTIFF → JPG dönüşüm
src_dir = "./spacenet/AOI_2_Vegas_Train/RGB-PanSharpen"
dst_dir = "./spacenet/RGB-Converted"
label_dir = "./spacenet/labels"
os.makedirs(dst_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)

converted, skipped = 0, 0
for file in os.listdir(src_dir):
    if file.endswith('.tif'):
        base = os.path.splitext(file)[0]
        label_path = os.path.join(label_dir, f"{base}.txt")
        if not os.path.exists(label_path) or os.path.getsize(label_path) == 0:
            skipped += 1
            continue
        try:
            with rasterio.open(os.path.join(src_dir, file)) as src:
                rgb = src.read([1, 2, 3])
                img = reshape_as_image(rgb)
                Image.fromarray(img.astype(np.uint8)).save(os.path.join(dst_dir, f"{base}.jpg"))
                converted += 1
        except Exception as e:
            print(f"⛔ {file} dönüştürülemedi: {e}")
            skipped += 1

print(f"✅ JPG dönüştürüldü: {converted}")
print(f"🚫 Atlanan dosya: {skipped}")

final_jpgs = [f for f in os.listdir(dst_dir) if f.endswith('.jpg')]
print(f"📷 Kullanılabilir JPG sayısı: {len(final_jpgs)}")

# 🧭 GeoJSON → YOLO etiket dönüşümü
def geojson_to_yolo(geojson_path, image_size, yolo_dir):
    gdf = gpd.read_file(geojson_path)
    os.makedirs(yolo_dir, exist_ok=True)
    base_name = os.path.splitext(os.path.basename(geojson_path))[0]
    with open(f"{yolo_dir}/{base_name}.txt", 'w') as f:  # 'a' yerine 'w' kullanıldı
        for _, row in gdf.iterrows():
            if row.geometry.geom_type == "Polygon":
                minx, miny, maxx, maxy = row.geometry.bounds
                x_center = ((minx + maxx) / 2) / image_size[0]
                y_center = ((miny + maxy) / 2) / image_size[1]
                width = (maxx - minx) / image_size[0]
                height = (maxy - miny) / image_size[1]
                f.write(f"0 {x_center} {y_center} {width} {height}\n")

geojson_dir = "./spacenet/AOI_2_Vegas_Train/geojson/buildings"
if final_jpgs:
    sample_path = os.path.join(dst_dir, final_jpgs[0])
    with Image.open(sample_path) as img:
        image_size = img.size

    # Düzeltilen kısım: eksik parametreler eklendi ve parantez kapatıldı
    for geojson_file in os.listdir(geojson_dir):
        if geojson_file.endswith(".geojson"):
            geojson_to_yolo(os.path.join(geojson_dir, geojson_file), image_size, label_dir)

    print(f"✅ GeoJSON → YOLO dönüşümü tamamlandı!")
else:
    print("❌ JPG dosyası bulunamadı, etiket dönüşümü yapılamadı.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21